# Pobranie bazy danych przy użyciu Kaggle API


In [44]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import SGD

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

# Moje API
kaggle_json = {
    "username": "miczelzieb",
    "key": "854eca6e83763f817886d45e3daa0c4"
}

# Tworze katalog
os.makedirs(os.path.expanduser("~/.kaggle"), exist_ok=True)

# Zapisuje plik
with open(os.path.expanduser("~/.kaggle/kaggle.json"), "w") as f:
    json.dump(kaggle_json, f)
# Zmieniam uprawnienia
os.chmod(os.path.expanduser("~/.kaggle/kaggle.json"), 0o600)


# OOP Klasa KaggleDatasetFetcher pobiera nazwę datasetu z Kaggle i importuje ją do folderu data na komputerze


In [71]:
class KaggleDatasetFetcher:
    def __init__(self, dataset_name, download_path='data/'):
        self.dataset_name = dataset_name
        self.download_path = download_path
        self.api = KaggleApi()
        self.api.authenticate()

    def download_dataset(self):
        if not os.path.exists(self.download_path):
            os.makedirs(self.download_path)

        print(f"Pobieranie: {self.dataset_name}")
        self.api.dataset_download_files(self.dataset_name, path=self.download_path, unzip=True)
        print(f"Baza pobrana do: {self.download_path}")

if __name__ == "__main__":
    dataset_name = 'kazanova/sentiment140'
    fetcher = KaggleDatasetFetcher(dataset_name)
    fetcher.download_dataset()


Pobieranie: kazanova/sentiment140
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
Baza pobrana do: data/


# Import i wybieranie kolumn

In [7]:
baza = pd.read_csv("data/training.1600000.processed.noemoticon.csv", encoding='latin1')

In [8]:
baza.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [9]:
mapowania = {"0": "sentyment", "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": "tweet"}
baza.rename(columns = mapowania, inplace = True)
baza.head(2)

,sentyment,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,tweet
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...


In [10]:
baza = baza[["tweet", "sentyment"]]

In [11]:
baza.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   tweet      1599999 non-null  object
 1   sentyment  1599999 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


# Wstępna analiza danych

In [12]:
tweets = baza["tweet"]
num_tweets = len(tweets)
tweet_lengths = tweets.apply(len)
word_counts = tweets.apply(lambda x: len(x.split()))

In [13]:
print("Podstawowe statystyki dotyczące tweetów:")
print(f"Liczba tweetów: {num_tweets}")
print(f"Średnia długość tweetów: {np.mean(tweet_lengths)}")
print(f"Mediana długości tweetów: {np.median(tweet_lengths)}")
print(f"Minimalna długość tweetów: {np.min(tweet_lengths)}")
print(f"Maksymalna długość tweetów: {np.max(tweet_lengths)}")
print(f"Odchylenie standardowe długości tweetów: {np.std(tweet_lengths)}")
print(f"Średnia liczba słów w tweetach: {np.mean(word_counts)}")
print(f"Mediana liczby słów w tweetach: {np.median(word_counts)}")
print(f"Minimalna liczba słów w tweetach: {np.min(word_counts)}")
print(f"Maksymalna liczba słów w tweetach: {np.max(word_counts)}")
print(f"Odchylenie standardowe liczby słów w tweetach: {np.std(word_counts)}")

Podstawowe statystyki dotyczące tweetów:
Liczba tweetów: 1599999
Średnia długość tweetów: 74.09008568130355
Mediana długości tweetów: 69.0
Minimalna długość tweetów: 6
Maksymalna długość tweetów: 374
Odchylenie standardowe długości tweetów: 36.44112440186655
Średnia liczba słów w tweetach: 13.176146985091865
Mediana liczby słów w tweetach: 12.0
Minimalna liczba słów w tweetach: 1
Maksymalna liczba słów w tweetach: 64
Odchylenie standardowe liczby słów w tweetach: 6.957976853399024


# Normalizacja tekstu + stemming + lematyzacja


In [14]:
# Tokenizacja tekstu
tokenized_tweets = tweets.apply(word_tokenize)

# Usuwanie znaków specjalnych
tokenized_tweets = tokenized_tweets.apply(lambda x: [word for word in x if word.isalnum()])

# Zamiana na małe litery
tokenized_tweets = tokenized_tweets.apply(lambda x: [word.lower() for word in x])

# Usuwanie stop words
stop_words = set(stopwords.words('english'))
tokenized_tweets = tokenized_tweets.apply(lambda x: [word for word in x if word not in stop_words])

# Lematyzacja
lemmatizer = WordNetLemmatizer()
tokenized_tweets = tokenized_tweets.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Usuwanie tokenów o długości 1
tokenized_tweets = tokenized_tweets.apply(lambda x: [word for word in x if len(word) > 1])

# Konwersja tokenów z powrotem na tekst
normalized_tweets = tokenized_tweets.apply(lambda x: ' '.join(x))

In [15]:
for i in range(5):
  print(normalized_tweets[i])

upset ca update facebook texting might cry result school today also blah
kenichan dived many time ball managed save 50 rest go bound
whole body feel itchy like fire
nationwideclass behaving mad ca see
kwesidei whole crew


In [63]:
# Podział danych na trzy grupy według wartości etykiety sentymentu
negative_group = dane[dane['sentyment'] == 0]
neutral_group = dane[dane['sentyment'] == 2]
positive_group = dane[dane['sentyment'] == 4]

# Pobranie 33% obserwacji z każdej grupy
negative_sample = negative_group.sample(frac=0.33, random_state=42)
neutral_sample = neutral_group.sample(frac=0.33, random_state=42)
positive_sample = positive_group.sample(frac=0.33, random_state=42)

# Połączenie próbek w jednym zestawie danych
sampled_data = pd.concat([negative_sample, neutral_sample, positive_sample], ignore_index=True)

# Wybór 10 000 obserwacji
sampled_data = sampled_data.sample(n=100000, random_state=42)

# Mapowanie wartości sentymentu do 0 i 1
sampled_data['sentyment'] = sampled_data['sentyment'].map({0: 0, 4: 1})

# Sprawdzenie rozkładu etykiet w pobranych danych
print(sampled_data['sentyment'].value_counts())

sentyment
0    50029
1    49971
Name: count, dtype: int64


In [64]:
X_train, X_test, y_train, y_test = train_test_split(sampled_data['tweet'], sampled_data['sentyment'], test_size=0.2, random_state=42)

# Wektoryzacja tekstu + modelowanie

In [65]:
# Tokenizacja tekstu
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

# Wyrównanie długości sekwencji
max_length = max([len(seq) for seq in X_train_sequences])
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

# Budowa modelu
model = Sequential([
    Embedding(input_dim=len(word_index)+1, output_dim=16, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Wyjście binarne dla dwóch klas
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Trenowanie modelu
history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

# Ocena modelu
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Epoch 1/10
2500/2500 [==============================] - 57s 20ms/step - loss: 0.5266 - accuracy: 0.7303 - val_loss: 0.4895 - val_accuracy: 0.7628
Epoch 2/10
2500/2500 [==============================] - 35s 14ms/step - loss: 0.3695 - accuracy: 0.8377 - val_loss: 0.5403 - val_accuracy: 0.7363
Epoch 3/10
2500/2500 [==============================] - 36s 14ms/step - loss: 0.2669 - accuracy: 0.8878 - val_loss: 0.6288 - val_accuracy: 0.7384
Epoch 4/10
2500/2500 [==============================] - 34s 14ms/step - loss: 0.2064 - accuracy: 0.9138 - val_loss: 0.6660 - val_accuracy: 0.7319
Epoch 5/10
2500/2500 [==============================] - 37s 15ms/step - loss: 0.1575 - accuracy: 0.9350 - val_loss: 0.8117 - val_accuracy: 0.7260
Epoch 6/10
2500/2500 [==============================] - 35s 14ms/step - loss: 0.1242 - accuracy: 0.9489 - val_loss: 1.0263 - val_accuracy: 0.7289
Epoch 7/10
2500/2500 [==============================] - 34s 14ms/step - loss: 0.1000 - accuracy: 0.9595 - val_loss: 1.0437 -

# Test jednostkowy, który sprawdza, czy dokładność zbioru testowego jest większa niż 70%

In [ ]:
def test_sentiment_model():
    # Tokenizacja tekstu
    tokenizer = Tokenizer(oov_token='<OOV>')
    tokenizer.fit_on_texts(X_train)
    X_train_sequences = tokenizer.texts_to_sequences(X_train)
    X_test_sequences = tokenizer.texts_to_sequences(X_test)

    # Wyrównanie długości sekwencji
    max_length = max([len(seq) for seq in X_train_sequences])
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding='post')
    X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length, padding='post')

    # Budowa modelu
    model = Sequential([
        Embedding(input_dim=len(word_index)+1, output_dim=16, input_length=max_length),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')  # Wyjście binarne dla dwóch klas
    ])
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Trenowanie modelu
    history = model.fit(X_train_padded, y_train, epochs=10, validation_data=(X_test_padded, y_test))

    # Ocena modelu
    test_loss, test_acc = model.evaluate(X_test_padded, y_test)

    # Sprawdzenie, czy dokładność testowa jest większa od 70%
    assert test_acc > 0.7, "Test accuracy less than 70%"

# Uruchomienie testu
test_sentiment_model()

# Analiza i ocena wyników

In [70]:
test_loss, test_acc = model.evaluate(X_test_padded, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

625/625 [==============================] - 3s 5ms/step - loss: 1.3488 - accuracy: 0.7110
Test Loss: 1.348825216293335
Test Accuracy: 0.7110000252723694


## Model sieci neuronowej ma zadawalające parametry przy teście na zbiorze testowym (71%), ale z pewnością wystąpiło przetrenowanie (97% zbiór treningowy vs 71% zbiór uczący). Na pewno jest możliwe poprawa tego wynik np. poprzez pogłębienie sieci bądź zastosowanie regularyzacji.